# Hyperparameter search

### Importing our own functions

In [38]:
import importlib

import src.Data as Data
importlib.reload(Data)
Data = Data.Data

import src.Metrics as Metrics
importlib.reload(Metrics)
Metrics = Metrics.Metrics

### Importing libraries

In [39]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import LearningCurveDisplay, learning_curve, cross_validate, train_test_split, cross_val_predict
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import pandas as pd

### Loading the data

In [40]:
data: Data = Data(test_size=0.2, include_images=False)

least_populated_class_count = np.unique(data.y_train, return_counts=True)[1].min()
print("Least populated class count:", least_populated_class_count)
print("This is the maximum valid number of folds for cross validation.")

Least populated class count: 8
This is the maximum valid number of folds for cross validation.


### Choosing the models

In [41]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier

classifiers = [
    # DecisionTreeClassifier(), 
    # RandomForestClassifier(n_jobs=-1), 
    # BaggingClassifier(n_jobs=-1), 
    # LogisticRegression(n_jobs=-1), 
    # SVC(), 
    # GaussianNB(), 
    # SGDClassifier(n_jobs=-1), 
    # KNeighborsClassifier(n_jobs=-1), 
    GradientBoostingClassifier(), 
    # MLPClassifier(), 
    AdaBoostClassifier()
]

### Getting the list of hyperparameters


In [42]:
for classifier in classifiers:
    print("Classifier:", classifier.__class__.__name__)
    print("Parameters:")
    for key in classifier.get_params():
        print("\t", key)
    print("")
    

Classifier: GradientBoostingClassifier
Parameters:
	 ccp_alpha
	 criterion
	 init
	 learning_rate
	 loss
	 max_depth
	 max_features
	 max_leaf_nodes
	 min_impurity_decrease
	 min_samples_leaf
	 min_samples_split
	 min_weight_fraction_leaf
	 n_estimators
	 n_iter_no_change
	 random_state
	 subsample
	 tol
	 validation_fraction
	 verbose
	 warm_start

Classifier: AdaBoostClassifier
Parameters:
	 algorithm
	 base_estimator
	 estimator
	 learning_rate
	 n_estimators
	 random_state

